Simulate doublets from snRNA-seq of rats with corresponding genotype data available.

**Samples**

| Sample | Description | cells in BAM | cells in filtered | cells in filtered after removing duplicate barcodes |
|------|------|------|------| ------|
| Rat_Opioid_HS_1 | cocaine_low | 311142 | 9389 | 9311 |
| Rat_Opioid_HS_2 | cocaine_high | 325968 | 8372 | 8299 |
| Rat_Opioid_HS_3 | oxycodone_low | 298371 | 7088 | 7023 |
| Rat_Opioid_HS_4 | oxycodone_high | 309449 | 8415 | 8340 |
| Rat_Amygdala_787A_all_seq | naive | 348129 | 5422 | 5375 |

Total cells from BAM files: 1593059

Total cells from filtered: 38686

**Total cells from filtered after removing duplicates: 38348**

Given an estimated doublet rate $d$, the total number of cells $x$, and the number of samples $N$, we will estimate the total number of doublets as $D = \frac{d x}{2}$.

Kang et al. defines the probability that a doublet contains cells from different individuals as $1-1/N$. Given this probability and the estimated number of doublets, $D$, we will define the estimated number of doublets that contain cells from different individuals as $a = \lfloor D(1-1/N) \rceil$ and the number of doublets that contain cells from the same individual as $b = D-a$.

We will shuffle the lists of barcodes from each sample to begin and store each in a list. 

We will randomly choose $b$ times with replacement from an array of 5 values to represent the samples from which each doublet containing cells from the same individual comes. 

For doublets containing cells from different individuals, we will randomly choose a 2x$a$ sized array from an array of 5 values to represent the two samples from which each doublet containing cells from different individuals comes.

For each doublet, we will pop a barcodes from the list of barcodes corresponding to a sample to generate pairs of cells that will become simulated doublets. 

##### Reassigning cell barcodes for simulated doublets
For each of these doublets, we will assign one member of the doublet the same cell barcode as the other member of the doublet. The output of this task will generate a table mapping cells with changed barcodes to the new barcode.

In [1]:
import numpy as np
import gzip
import glob
import random
from scipy import stats

Load filtered unique barcodes from each sample

In [2]:
barcode_files = glob.glob('/iblm/netapp/data1/jezhou/Telese_Rat_Amygdala/demultiplex_simulation/unique_filtered_barcodes/*')
sample_names = []
barcodes = []

for file in barcode_files:
    sample = file.split('.')[0].split('/')[-1]
    sample_names.append(sample)
    with gzip.open(file) as fh:
        sample_barcodes = fh.read().splitlines()
#         sample_barcodes = [bc.decode('utf-8') for bc in sample_barcodes]
        barcodes.append(sample_barcodes)

In [385]:
x = len([x for l in barcodes for x in l])
N = len(sample_names)

print('x = %d\nN = %d' % (x, N))

x = 38348
N = 5


Set the doublet rate and compute $D,a,b$. 

We will set the doublet rate as $d=0.3$. This means that 30% of all of the cells will end up in doublets.

In [386]:
d = 0.3

totDoublets = int(round(d*x/2))
a = int(round(totDoublets*(1-1/N)))
b = totDoublets-a

print('D = %d\na = %d\nb = %d' % (totDoublets, a, b))

D = 5752
a = 4602
b = 1150


Shuffle barcodes for each sample

In [387]:
shuffled_barcodes = [random.sample(bc, len(bc)) for bc in barcodes]

Simulate doublets containing cells from the same sample

In [388]:
weights = [len(bc)/x for bc in barcodes]
weights

[0.24280275372900803,
 0.2164128507353708,
 0.18313862522165433,
 0.2174820068843225,
 0.14016376342964432]

In [389]:
doublets_same = np.unique(np.random.choice(5, b, p = weights), return_counts = True)

In [390]:
simulated_doublets_same = []
for sample, counts in zip(doublets_same[0], doublets_same[1]):
    print(sample)
    print(counts)
    for i in range(counts):
        bcA = shuffled_barcodes[sample].pop()
        bcB = shuffled_barcodes[sample].pop()
        doublet = [bcA, bcB]
        simulated_doublets_same.append(doublet)

0
284
1
267
2
214
3
225
4
160


Simulate doublets containing cells from different samples

In [399]:
doublets_diff = []
for i in range(a):
    dbl = np.random.choice(5,2,p = weights,replace = False)
    doublets_diff.append(dbl)

doublets_diff = np.array(doublets_diff)

In [400]:
simulated_doublets_diff = []
for pair in doublets_diff:
    bcA = shuffled_barcodes[pair[0]].pop()
    bcB = shuffled_barcodes[pair[1]].pop()
    doublet = [bcA, bcB]
    simulated_doublets_diff.append(doublet)

Change barcodes such that cells in doublet have same barcodes.

In [401]:
# make dict mapping second barcode in each doublet to first barcode
new_barcodes_map = {}
for doublet in simulated_doublets_same + simulated_doublets_diff:
    new_barcodes_map[doublet[1]] = doublet[0]

In [402]:
# generate new filtered.tsv.gz files
out_dir = '/iblm/netapp/data1/jezhou/Telese_Rat_Amygdala/demultiplex_simulation/renamed_filtered_barcodes'

for sample,bcs in zip(sample_names, barcodes):
    print(sample)
    new_bcs = []
    mismatches = 0
    for i in range(len(bcs)):
        if bcs[i] in new_barcodes_map.keys():
            new_bcs.append(new_barcodes_map[bcs[i]])
            mismatches += 1
        else:
            new_bcs.append(bcs[i])
    print(mismatches)
    new_bcs = sorted(new_bcs)
    with gzip.open('%s/%s.tsv.gz' % (out_dir, sample), 'wb') as fh:
        for bc in new_bcs:
            fh.write(bc)
            fh.write('\n'.encode('utf-8'))
        fh.close()

Rat_Opioid_HS_1_premrna
1380
Rat_Opioid_HS_2_premrna
1258
Rat_Opioid_HS_3_premrna
1105
Rat_Opioid_HS_4_premrna
1157
Rat_Amygdala_787A_all_seq
852


In [403]:
# check that number of mismatches is as expected
doublets_same[1] + np.unique(doublets_diff[:,1], return_counts = True)[1]

array([1380, 1258, 1105, 1157,  852])

Write list of doublets that come from the same cell versus different cells

In [404]:
out_dir = '/iblm/netapp/data1/jezhou/Telese_Rat_Amygdala/demultiplex_simulation'

# write doublets with cells from same cells
doublets_same_reference = np.concatenate((np.repeat(sample_names, doublets_same[1]).reshape(-1,1),
               np.array(simulated_doublets_same)), axis = 1)

with open(out_dir + '/same_sample_doublets_reference.tsv', 'w') as fh:
    for l in doublets_same_reference:
        fh.write('\t'.join(l) + '\n')

In [406]:
# map doublets_diff to sample names
map_samples = dict(zip(range(5), sample_names))
doublets_diff_with_sample_names = []
for i in range(doublets_diff.shape[0]):
    sampleA = map_samples[doublets_diff[i,0]]
    sampleB = map_samples[doublets_diff[i,1]]
    doublets_diff_with_sample_names.append([sampleA, sampleB])

doublets_diff_reference = np.concatenate((doublets_diff_with_sample_names, simulated_doublets_diff), axis = 1)

with open(out_dir + '/diff_sample_doublets_reference.tsv', 'w') as fh:
    for l in doublets_diff_reference:
        fh.write('\t'.join(l) + '\n')